# CNN Image Classification — Rock Paper Scissors

Notebook generated from `pes2ug23cs348_CNN.pdf`. 

**Author:** Mohammedshazi

**Notes:** Update `DATA_DIR` to point to your dataset path before running cells.

## 1. Setup and Data Download

This notebook expects the Rock-Paper-Scissors dataset to be available in `DATA_DIR`.
If using Kaggle, you can download `drgfreeman/rockpaperscissors` and extract to `DATA_DIR`.

Default `DATA_DIR = '/content/dataset'`.

In [ ]:
# Imports and device setup
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# Data loading and preprocessing
DATA_DIR = "/content/dataset"  # <- Change this path if your dataset is stored elsewhere

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])

full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
class_names = full_dataset.classes
print("Classes:", class_names)
total = len(full_dataset)
train_size = int(0.8 * total)
test_size = total - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Test images: {len(test_dataset)}")

In [ ]:
# Define the CNN model
class RPS_CNN(nn.Module):
    def __init__(self):
        super(RPS_CNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 3),
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x

model = RPS_CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

In [ ]:
# Training loop (10 epochs by default)
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss = {avg_loss:.4f}")
print("Training complete!")

In [ ]:
# Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
# Predict on single image
def predict_image(model, img_path, transform, class_names, device):
    model.eval()
    img = Image.open(img_path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        _, pred = torch.max(output, 1)
    return class_names[pred.item()]

In [ ]:
# Simple 'play' using model predictions on random images
import random
def pick_random_image(class_name, data_root=DATA_DIR):
    folder = os.path.join(data_root, class_name)
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    img = random.choice(files)
    return os.path.join(folder, img)

def rps_winner(move1, move2):
    if move1 == move2:
        return 'Draw'
    rules = {'rock':'scissors','paper':'rock','scissors':'paper'}
    if rules[move1] == move2:
        return f'Player 1 wins! {move1} beats {move2}'
    else:
        return f'Player 2 wins! {move2} beats {move1}'

In [ ]:
# Save the trained model
# torch.save(model.state_dict(), '/content/rps_cnn.pth')